# MAE 1001: Introduction to Mechanical and Aerospace Engineering 
# Dice in Bias Experiment
### Instructor: 
#### Prof. Kartik Bulusu [MAE]

**Teaching Assistant:** 

Preethi Siva Kumar [MAE-MS student]

**Learning Assistants:** 

Nathan Janssen [MAE-Senior], Olivia Falciani [MAE-Senior], Shota Kakiuichi [MAE-Senior]

**Technical Support**

Rithik Gunuganti [CS-MS student]


If you have any questions regarding Python, please feel free to send a slack message or
email to a member of the teaching team or come to office hours! We are happy to help!!

**Block 1:**
Import Libraries and Define Filename Function
What it does: Loads the tools (libraries) needed for the app and creates a function to name the CSV file based on the die and date.

**Explanation:**

***Libraries:*** These are like toolkits. dash creates the web app, plotly.express draws graphs, pandas manages data, os handles files, and datetime gets the current date.

***Function:*** get_csv_filename makes a filename for the CSV (e.g., p4_102425.csv) using the die name (like p_4) and today’s date (like 10/24/25).



In [1]:
!pip install -r requirements_MAE1001.txt
# Import libraries (tools) for the app
import dash  # Builds the web app
import pandas as pd  # Handles data tables
import plotly.express as px  # Makes histograms
import os  # Checks/saves files
from datetime import datetime  # Gets today's date
from dash import dcc, html, Input, Output, State  # Parts for the app's layout and actions

# Function to create CSV filename (e.g., p4_102325.csv)
def get_csv_filename(die_name):
    today = datetime.now().strftime('%m/%d/%y')  # Get date like 10/23/25
    clean_die = die_name.replace('_', '')  # Remove '_' from die name (p_4 -> p4)
    return f"{clean_die}_{today.replace('/', '')}.csv"  # Combine die name and date

**Block 2: Set Up the Dash App and Global Variables**

***What it does:*** Starts the Dash app and sets up a counter to track the number of rolls (up to 50).

**Explanation:**

***App Setup:*** dash.Dash(name) creates the web app. server prepares it to run online or locally.

***Variables:*** entry_count tracks how many rolls are submitted. MAX_ENTRIES sets the limit to 50 rolls.

In [2]:
# Start the Dash app
app = dash.Dash(__name__)  # Creates the app; __name__ is the current file
server = app.server  # For running the app (Added for future enhancement, currently the app is deployed locally)

# Counter to track rolls
entry_count = 0  # Starts at 0
MAX_ENTRIES = 50  # Stops at 50 rolls

**Block 3: Define the App Layout**

What it does: Creates the app’s webpage with input fields, buttons for face values (1–6), and a graph area.

**Explanation:**

***Layout:*** This sets up the webpage with text boxes for last name, first name, and die name, plus buttons for selecting face values (1–6).

***Components:*** ***html.H2*** and ***html.H3*** are titles. ***dcc.Input*** creates text boxes. ***html.Button*** makes clickable buttons. ***dcc.Graph*** shows the histogram. ***dcc.Store*** tracks button clicks invisibly.

***Style:*** The flex and gap styles arrange buttons neatly in a row.

In [3]:
# Create the app's webpage layout
app.layout = html.Div([
    html.H2("Dice Roll Experiment Entry"),  # Big title
    html.Div([  # Group for input fields
        html.Label('Last Name'),  # Text label
        dcc.Input(id='last_name', type='text', value=''),  # Text box for last name
        html.Br(),  # Line break
        html.Label('First Name'),
        dcc.Input(id='first_name', type='text', value=''),  # Text box for first name
        html.Br(),
        html.Label('Die Name (e.g., o_2 for orange die #2)'),
        dcc.Input(id='die_name', type='text', value=''),  # Text box for die name
        html.Br(),
        # Buttons for face values (1 to 6)
        html.Div([
            html.Label('Die Face Value'),
            html.Button('1', id='btn_1', n_clicks=0),  # Button for face value 1
            html.Button('2', id='btn_2', n_clicks=0),
            html.Button('3', id='btn_3', n_clicks=0),
            html.Button('4', id='btn_4', n_clicks=0),
            html.Button('5', id='btn_5', n_clicks=0),
            html.Button('6', id='btn_6', n_clicks=0),
        ], style={'display': 'flex', 'gap': '10px'}),  # Arrange buttons in a row
        html.Div(id='error_msg', style={'color': 'red'}),  # Shows error messages
        html.Div(id='status_msg', style={'color': 'green'}),  # Shows success messages
        html.Div(id='terminate_msg', style={'color': 'blue'}),  # Shows stop message
    ]),
    html.H3("Histogram of Rolls for This Die"),  # Title for graph
    dcc.Graph(id='histogram_plot'),  # Area for histogram
    dcc.Store(id='store_clicks', data={'btn_1': 0, 'btn_2': 0, 'btn_3': 0, 'btn_4': 0, 'btn_5': 0, 'btn_6': 0})  # Hidden storage for button clicks
])

**Block 4: Handle Button Clicks and Update the App**

***What it does:*** Processes button clicks, saves data to a CSV, updates the histogram, and shows messages.

***Explanation:***

***Callback:*** This function runs when a face value button (1–6) is clicked. It checks which button was pressed and gets the text box values.

***Button Logic:*** Uses dcc.Store to track button clicks and determine which face value (1–6) was selected.

***Data Handling:*** Saves roll data (name, die, face value) to a CSV file. Updates the roll counter.

***Histogram:*** Shows a bar graph of face values for the current die. If no data, shows an empty plot.

***Messages:*** Displays errors (red), success (green), or stop messages (blue).
Running: The final line starts the app, opening a webpage for interaction.

In [4]:
# Function to handle button clicks and update app
@app.callback(
    [Output('histogram_plot', 'figure'),  # Updates histogram
     Output('error_msg', 'children'),  # Updates error message
     Output('status_msg', 'children'),  # Updates success message
     Output('terminate_msg', 'children'),  # Updates stop message
     Output('store_clicks', 'data')],  # Updates button click storage
    [Input('btn_1', 'n_clicks'),  # Tracks clicks on buttons 1-6
     Input('btn_2', 'n_clicks'),
     Input('btn_3', 'n_clicks'),
     Input('btn_4', 'n_clicks'),
     Input('btn_5', 'n_clicks'),
     Input('btn_6', 'n_clicks')],
    [State('last_name', 'value'),  # Gets text box values
     State('first_name', 'value'),
     State('die_name', 'value'),
     State('store_clicks', 'data')]  # Gets stored button clicks
)
def update_output(btn_1, btn_2, btn_3, btn_4, btn_5, btn_6, last_name, first_name, die_name, clicks_data):
    global entry_count  # Use global counter
    error = ""  # Empty error message
    status = ""  # Empty success message
    terminate = ""  # Empty stop message
    
    # Check which button was clicked
    face_value = None
    if btn_1 > clicks_data['btn_1']:
        face_value = 1
    elif btn_2 > clicks_data['btn_2']:
        face_value = 2
    elif btn_3 > clicks_data['btn_3']:
        face_value = 3
    elif btn_4 > clicks_data['btn_4']:
        face_value = 4
    elif btn_5 > clicks_data['btn_5']:
        face_value = 5
    elif btn_6 > clicks_data['btn_6']:
        face_value = 6

    # Update stored button clicks
    clicks_data = {'btn_1': btn_1, 'btn_2': btn_2, 'btn_3': btn_3, 'btn_4': btn_5, 'btn_5': btn_5, 'btn_6': btn_6}

    if face_value is None:
        # No button clicked yet, show empty plot
        empty_df = pd.DataFrame({'Face Value': []})  # Empty table
        fig = px.histogram(empty_df, x='Face Value', nbins=6, title='No data yet!')
        fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False})
        return fig, error, status, terminate, clicks_data

    # Check if all fields are filled
    if not (last_name and first_name and die_name and face_value):
        error = "Please fill out all fields."
        df = pd.DataFrame()  # Empty table
    else:
        try:
            if face_value < 1 or face_value > 6:
                error = "Face value must be between 1 and 6."
            elif entry_count >= MAX_ENTRIES:
                error = "Maximum 50 entries reached. Please stop the app."
                terminate = "App will not accept more entries. Close the browser tab or stop the script."
            else:
                # Get CSV filename
                data_file = get_csv_filename(die_name)
                # Load or create data table
                if os.path.exists(data_file):
                    df = pd.read_csv(data_file)  # Read existing CSV
                else:
                    df = pd.DataFrame(columns=['Last Name', 'First Name', 'Die Name', 'Face Value'])  # New table

                # Add new roll to table
                new_row = {
                    'Last Name': last_name,
                    'First Name': first_name,
                    'Die Name': die_name,
                    'Face Value': face_value
                }
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                df.to_csv(data_file, index=False)  # Save to CSV

                # Update counter
                entry_count += 1
                status = f"Roll submitted! Total entries: {entry_count}. (Min 10 rolls recommended)"

                if entry_count >= MAX_ENTRIES:
                    terminate = "Reached 50 entries. No more submissions allowed. Stop the app and send the CSV file."

        except ValueError:
            error = "Face value must be an integer."

    # Show histogram
    if not df.empty:
        fig = px.histogram(df, x='Face Value', nbins=6,
                           title=f"Histogram of Face Values for Die: {die_name}",
                           labels={'Face Value': 'Face Value (1-6)'},
                           category_orders={'Face Value': [1, 2, 3, 4, 5, 6]})
        fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))  # Style bars
        fig.update_layout(bargap=0.1)  # Add gap between bars
    else:
        empty_df = pd.DataFrame({'Face Value': []})
        fig = px.histogram(empty_df, x='Face Value', nbins=6, title='No data yet!')
        fig.update_layout(xaxis={'visible': False}, yaxis={'visible': False})

    return fig, error, status, terminate, clicks_data

# Run the app
if __name__ == '__main__':
    app.run(debug=False)